In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

!pip install torch_geometric torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__.split('+')[0])")+cpu.html
!pip install trimesh matplotlib


import os
import torch
import torch.nn.functional as F
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import trimesh
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


class NOFFDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.files = []
        for dirpath, _, filenames in os.walk(root):
            for f in filenames:
                if f.endswith('.off') or f.endswith('.noff'):
                    self.files.append(os.path.join(dirpath, f))
        self.cache = {}

    def len(self):
        return len(self.files)

    def get(self, idx):
        if idx in self.cache:
            return self.cache[idx]

        mesh = trimesh.load(self.files[idx], process=False)

        x = torch.tensor(mesh.vertices, dtype=torch.float)

        edges = []
        for face in mesh.faces:
            edges.append([face[0], face[1]])
            edges.append([face[1], face[2]])
            edges.append([face[2], face[0]])
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        y = torch.tensor([0], dtype=torch.float)

        data = Data(x=x, edge_index=edge_index, y=y)
        self.cache[idx] = data
        return data


dataset_path = "/content/drive/MyDrive/car"
dataset = NOFFDataset(root=dataset_path)

print("Dataset size:", len(dataset))
print(dataset[0])



class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin = torch.nn.Linear(hidden_dim, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.lin(x).squeeze(-1)
        return x



loader = DataLoader(dataset, batch_size=1, shuffle=True)
model = GCN(input_dim=3, hidden_dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 6):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = F.binary_cross_entropy_with_logits(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}, Loss: {total_loss:.4f}")



model.eval()
data = dataset[0].to(device)
data.batch = torch.zeros(data.num_nodes, dtype=torch.long, device=device)

with torch.no_grad():
    x1 = F.relu(model.conv1(data.x, data.edge_index))
    x2 = F.relu(model.conv2(x1, data.edge_index))

importance = x2.norm(dim=1).cpu().numpy()

mesh = trimesh.load(dataset.files[0], process=False)
colors = plt.cm.jet((importance - importance.min()) / (importance.max() - importance.min()))[:, :3]
mesh.visual.vertex_colors = (colors * 255).astype(np.uint8)
mesh.show()

Looking in links: https://data.pyg.org/whl/torch-2.8.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.6/645.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.6/749.6 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.4/289.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.5/736.5 kB 10.7 MB/s eta 0:00:00
Using device: cpu
Dataset size: 297
Data(x=[49847, 3], edge_index=[2, 82128], y=[1])


/tmp/ipython-input-2720567956.py:78: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  loader = DataLoader(dataset, batch_size=1, shuffle=True)


Epoch 1, Loss: 5.8888
Epoch 2, Loss: 0.0030
Epoch 3, Loss: 0.0010
Epoch 4, Loss: 0.0005
Epoch 5, Loss: 0.0003
